### Show the position of ATP of predicted result

In [9]:
import os
import glob
import pandas as pd

# 指定你的文件夹路径
folder_path = 'predicted_test'  # 替换成你的文件夹路径

# 使用 glob 获取文件夹下所有 CSV 文件的完整路径
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# 创建一个列表保存每个文件的结果
results = []

# 遍历每个 CSV 文件
for file in csv_files:
    df = pd.read_csv(file)
    
    # 根据 'ATPseq Binding Sites' 列筛选出预测为ATP结合位点的行（这里假设标记 'B' 表示结合位点）
    atpseq_binding = df[df['ATPseq Binding Sites'] == 'B']
    
    # 提取位置编号（假设存储在 'NO' 列中），也可以根据需求提取其它信息
    positions = atpseq_binding['NO'].tolist()
    
    # 保存结果：可以保存文件名、预测的位点位置及个数等信息
    results.append({
        'filename': os.path.basename(file),
        'binding_positions': positions,
        'num_binding': len(positions)
    })

# 输出每个文件的结果，便于比较
for res in results:
    print(f"文件：{res['filename']}, 预测的ATP结合位点数量：{res['num_binding']}, 位置：{res['binding_positions']}")


文件：1mb9_chainB.csv, 预测的ATP结合位点数量：8, 位置：[243, 244, 246, 247, 248, 249, 266, 267]
文件：2j9c_chainA.csv, 预测的ATP结合位点数量：12, 位置：[9, 37, 38, 39, 41, 60, 88, 89, 90, 91, 92, 105]


### Show the position of ATP of experimental data

In [8]:
import os
import glob
from Bio.PDB import PDBParser, NeighborSearch
from Bio.PDB.Polypeptide import is_aa

# 指定存放 PDB 文件的文件夹路径（请替换为你的实际路径）
folder_path = 'raw_test'

# 获取文件夹下所有 pdb 文件
pdb_files = glob.glob(os.path.join(folder_path, '*.pdb'))

# 初始化 PDBParser
parser = PDBParser(QUIET=True)

# 设置距离阈值（单位：Å）
threshold = 4.0

for pdb_file in pdb_files:
    structure = parser.get_structure(os.path.basename(pdb_file), pdb_file)

    # 查找所有 ATP 配体的原子（ATP一般以 HETATM 记录出现）
    atp_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.get_resname() == "ATP":
                    for atom in residue:
                        atp_atoms.append(atom)

    # 如果没有找到 ATP 配体，则认为无结核点位
    if not atp_atoms:
        print(f"文件：{os.path.basename(pdb_file)}, 无结核点位（未找到 ATP 配体）。")
        continue

    # 收集所有标准氨基酸的原子
    protein_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if is_aa(residue, standard=True):
                    protein_atoms.extend(list(residue.get_atoms()))

    # 构建邻域搜索树
    ns = NeighborSearch(protein_atoms)

    # 存储 ATP 结合点位对应的残基信息，格式为 (chain_id, residue_id, resname)
    binding_residues = set()
    for atp_atom in atp_atoms:
        close_atoms = ns.search(atp_atom.get_coord(), threshold)
        for atom in close_atoms:
            residue = atom.get_parent()
            if is_aa(residue, standard=True):
                chain_id = residue.get_parent().get_id()
                res_id = residue.get_id()  # 格式为 (hetfield, 序号, insertion code)
                binding_residues.add((chain_id, res_id, residue.get_resname()))

    # 提取所有结合点的残基序号（忽略链信息和三字母代码）
    positions = sorted([res_id[1] for _, res_id, _ in binding_residues])

    filename = os.path.basename(pdb_file)
    if binding_residues:
        count = len(binding_residues)
        print(f"文件：{filename}, 预测的ATP结合点位数量：{count}, 位置：{positions}")
    else:
        print(f"文件：{filename}, 无结核点位")


文件：1mb9_chainB.pdb, 预测的ATP结合点位数量：18, 位置：[247, 248, 249, 251, 252, 253, 254, 271, 272, 273, 330, 333, 346, 347, 348, 351, 423, 443]
文件：2j9c_chainA.pdb, 预测的ATP结合点位数量：13, 位置：[7, 35, 36, 37, 38, 44, 58, 86, 87, 88, 89, 90, 92]


### Extract ATP position from PDB file from PDB data bank

In [6]:
import os
import glob
import csv
from Bio.PDB import PDBParser, NeighborSearch
from Bio.PDB.Polypeptide import is_aa, three_to_one

# 指定存放 PDB 文件的文件夹路径（请替换为你的实际路径）
input_folder = 'extracted_chains'  # 输入PDB文件夹路径
# 指定存放生成的 CSV 文件的文件夹路径
output_folder = 'ATP_extracted'  # 输出CSV文件夹路径
os.makedirs(output_folder, exist_ok=True)

# 获取输入文件夹下所有 pdb 文件
pdb_files = glob.glob(os.path.join(input_folder, '*.pdb'))

# 初始化 PDBParser
parser = PDBParser(QUIET=True)

# 设置距离阈值（单位：Å）
threshold = 4.0

for pdb_file in pdb_files:
    # 解析 PDB 文件
    structure = parser.get_structure(os.path.basename(pdb_file), pdb_file)
    # 使用文件名（去除扩展名）作为 Prot.ID
    prot_id_base = os.path.splitext(os.path.basename(pdb_file))[0]

    # 1. 提取所有 ATP 配体的原子（ATP通常以 HETATM 记录出现）
    atp_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.get_resname() == "ATP":
                    atp_atoms.extend(list(residue.get_atoms()))
    
    # 2. 利用 ATP 配体的原子，利用邻域搜索找出 ATP 结合点位
    binding_residues = set()  # 保存格式：(chain_id, residue_id, 三字母残基代码)
    if atp_atoms:
        # 收集所有标准氨基酸的原子
        protein_atoms = []
        for model in structure:
            for chain in model:
                for residue in chain:
                    if is_aa(residue, standard=True):
                        protein_atoms.extend(list(residue.get_atoms()))
        ns = NeighborSearch(protein_atoms)
        for atp_atom in atp_atoms:
            close_atoms = ns.search(atp_atom.get_coord(), threshold)
            for atom in close_atoms:
                residue = atom.get_parent()
                if is_aa(residue, standard=True):
                    chain_id = residue.get_parent().get_id()
                    res_id = residue.get_id()  # 格式为 (hetfield, 序号, insertion code)
                    binding_residues.add((chain_id, res_id, residue.get_resname()))
    
    # 3. 遍历所有标准氨基酸残基，生成 CSV 的每一行
    rows = []
    header = ['Prot.ID', 'NO', 'Residue', 'ATP Binding Site']
    rows.append(header)
    
    for model in structure:
        for chain in model:
            chain_id = chain.get_id()
            for residue in chain:
                if not is_aa(residue, standard=True):
                    continue
                res_id = residue.get_id()  # (het, seq_number, insertion code)
                # 转换三字母代码为一字母代码
                try:
                    res_one = three_to_one(residue.get_resname())
                except Exception as e:
                    res_one = residue.get_resname()
                # 判断是否为 ATP 结合点位
                if (chain_id, res_id, residue.get_resname()) in binding_residues:
                    binding_flag = "B"
                else:
                    binding_flag = "N"
                row = [prot_id_base, res_id[1], res_one, binding_flag]
                rows.append(row)
    
    # 4. 写入 CSV 文件，文件名与 PDB 文件名对应
    output_file = os.path.join(output_folder, prot_id_base + '.csv')
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(rows)
    
    print(f"生成文件: {output_file}")

print("批量处理完成！")


生成文件: ATP_extracted\121p_chainA.csv
生成文件: ATP_extracted\1d4x_chainA.csv
生成文件: ATP_extracted\1f3f_chainC.csv
生成文件: ATP_extracted\1fit_chainA.csv
生成文件: ATP_extracted\1i58_chainA.csv
生成文件: ATP_extracted\1j09_chainA.csv
生成文件: ATP_extracted\1k90_chainA.csv
生成文件: ATP_extracted\1mb9_chainB.csv
生成文件: ATP_extracted\1rn8_chainA.csv
生成文件: ATP_extracted\1s1d_chainA.csv
生成文件: ATP_extracted\1to6_chainA.csv
生成文件: ATP_extracted\1twf_chainB.csv
生成文件: ATP_extracted\1un9_chainA.csv
生成文件: ATP_extracted\1vl1_chainA.csv
生成文件: ATP_extracted\1wc6_chainB.csv
生成文件: ATP_extracted\1xdn_chainA.csv
生成文件: ATP_extracted\1xdp_chainA.csv
生成文件: ATP_extracted\1yzy_chainA.csv
生成文件: ATP_extracted\1z0s_chainA.csv
生成文件: ATP_extracted\2aqx_chainB.csv
生成文件: ATP_extracted\2bz0_chainA.csv
生成文件: ATP_extracted\2f17_chainA.csv
生成文件: ATP_extracted\2i1o_chainA.csv
生成文件: ATP_extracted\2j9c_chainA.csv
生成文件: ATP_extracted\2jg1_chainC.csv
生成文件: ATP_extracted\2py7_chainX.csv
生成文件: ATP_extracted\2q16_chainB.csv
生成文件: ATP_extracted\2x14_cha

### compare residue

In [14]:
import os
import glob
import pandas as pd

# 真值CSV文件存放文件夹（例如提取后的文件）
gt_folder = 'ATP_extracted'
# 预测结果CSV文件存放文件夹
pred_folder = 'predicted_result'

# 获取真值文件夹下所有 CSV 文件
gt_files = glob.glob(os.path.join(gt_folder, '*.csv'))

# 遍历每个真值文件
for gt_file in gt_files:
    base_name = os.path.basename(gt_file)
    pred_file = os.path.join(pred_folder, base_name)
    
    if not os.path.exists(pred_file):
        print(f"预测结果文件不存在: {pred_file}")
        continue

    # 读取文件
    gt_df = pd.read_csv(gt_file)
    pred_df = pd.read_csv(pred_file)
    
    # 获取Residue列，转换为列表
    gt_residues = list(gt_df['Residue'])
    pred_residues = list(pred_df['Residue'])
    
    # 比较长度
    if len(gt_residues) != len(pred_residues):
        print(f"文件 {base_name}: 长度不匹配 -> 真值长度: {len(gt_residues)}, 预测长度: {len(pred_residues)}")
    
    # 检查是否完全匹配（仅对最短长度范围内比对）
    min_len = min(len(gt_residues), len(pred_residues))
    mismatches = []
    for i in range(min_len):
        if gt_residues[i] != pred_residues[i]:
            mismatches.append((i, gt_residues[i], pred_residues[i]))
    
    if mismatches:
        print(f"文件 {base_name}: Residue不匹配:")
        for idx, gt_res, pred_res in mismatches:
            print(f"  索引 {idx}: 真值 = {gt_res}, 预测 = {pred_res}")
    else:
        #if len(gt_residues) == len(pred_residues):
            #print(f"文件 {base_name}: Residue完全匹配。")
        #else:
            # 即使在最小长度内都匹配，但长度不同也说明存在缺失
        print(f"文件 {base_name}: 最短范围内Residue匹配，但整体长度不一致。")


文件 121p_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1d4x_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1f3f_chainC.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1fit_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1i58_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1j09_chainA.csv: 长度不匹配 -> 真值长度: 469, 预测长度: 468
文件 1j09_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1k90_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1mb9_chainB.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1rn8_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1s1d_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1to6_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1twf_chainB.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1un9_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1vl1_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1wc6_chainB.csv: 长度不匹配 -> 真值长度: 193, 预测长度: 192
文件 1wc6_chainB.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1xdn_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1xdp_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1yzy_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1z0s_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 2aqx_chainB.csv: 最短范围内Residue匹配，但整体长度不一致。


### Compare and calculate TP...

In [16]:
import os
import glob
import pandas as pd
import math
from Bio import pairwise2

def align_labels(gt_residues, pred_residues, gt_binding, pred_binding):
    """
    对gt和pred的Residue序列进行全局比对，
    返回对齐后（两边均非gap）的真值结合标签和预测结合标签列表。
    """
    gt_seq = ''.join(gt_residues)
    pred_seq = ''.join(pred_residues)
    
    # 使用简单的全局比对（匹配得1分，不匹配得0分）
    alignments = pairwise2.align.globalxx(gt_seq, pred_seq)
    best = alignments[0]
    gt_aligned, pred_aligned = best.seqA, best.seqB

    aligned_gt_binding = []
    aligned_pred_binding = []
    i, j = 0, 0  # 指向原始序列的指针
    for a, b in zip(gt_aligned, pred_aligned):
        if a != '-' and b != '-':
            # 两边都有字符，则保存对应的结合标签
            aligned_gt_binding.append(gt_binding[i])
            aligned_pred_binding.append(pred_binding[j])
            i += 1
            j += 1
        elif a == '-' and b != '-':
            j += 1  # gt比对中出现缺失
        elif a != '-' and b == '-':
            i += 1  # pred比对中出现缺失
        else:
            # 同时为gap（很少见）
            pass
    return aligned_gt_binding, aligned_pred_binding

# 真值文件夹和预测结果文件夹（请修改为实际路径）
gt_folder = 'ATP_extracted'       # 存放提取后的真值CSV文件
pred_folder = 'predicted_result'          # 存放预测结果CSV文件

# 获取真值文件夹下所有CSV文件（假设每个蛋白对应一个CSV文件，且文件名一致）
gt_files = glob.glob(os.path.join(gt_folder, '*.csv'))

results = []

for gt_file in gt_files:
    base_name = os.path.basename(gt_file)
    pred_file = os.path.join(pred_folder, base_name)
    
    if not os.path.exists(pred_file):
        print(f"预测结果文件不存在: {pred_file}")
        continue

    # 读取真值和预测文件
    gt_df = pd.read_csv(gt_file)
    pred_df = pd.read_csv(pred_file)
    
    # 提取Residue序列（假设Residue列中的字符为一字母代码）
    gt_residues = list(gt_df['Residue'])
    pred_residues = list(pred_df['Residue'])
    
    # 提取真值的结合标签（假设列名为 "ATP Binding Site"，'B' 表示正，'N' 表示负）
    gt_binding = list(gt_df['ATP Binding Site'])
    
    # 对于两种预测方法分别计算指标
    for method, col in [('ATPseq', 'ATPseq Binding Sites'), ('ATPbind', 'ATPbind Binding Sites')]:
        pred_binding = list(pred_df[col])
        
        # 如果Residue序列不匹配，则使用序列对齐
        if gt_residues != pred_residues:
            aligned_gt_binding, aligned_pred_binding = align_labels(gt_residues, pred_residues, gt_binding, pred_binding)
        else:
            aligned_gt_binding = gt_binding
            aligned_pred_binding = pred_binding
        
        # 计算TP, FN, TN, FP（仅在对齐后的位置上计算）
        tp = sum(1 for gt, pred in zip(aligned_gt_binding, aligned_pred_binding) if gt == 'B' and pred == 'B')
        fn = sum(1 for gt, pred in zip(aligned_gt_binding, aligned_pred_binding) if gt == 'B' and pred == 'N')
        tn = sum(1 for gt, pred in zip(aligned_gt_binding, aligned_pred_binding) if gt == 'N' and pred == 'N')
        fp = sum(1 for gt, pred in zip(aligned_gt_binding, aligned_pred_binding) if gt == 'N' and pred == 'B')
        
        sen = tp / (tp + fn) if (tp + fn) != 0 else 0
        spe = tn / (tn + fp) if (tn + fp) != 0 else 0
        acc = (tp + tn) / (tp + fn + tn + fp) if (tp + fn + tn + fp) != 0 else 0
        pre = tp / (tp + fp) if (tp + fp) != 0 else 0
        denominator = math.sqrt((tp + fn) * (tp + fp) * (tn + fn) * (tn + fp))
        mcc = (tp * tn - fn * fp) / denominator if denominator != 0 else 0
        
        # 获取Prot.ID，假设真值文件中"Prot.ID"列的值都相同，取第一个
        prot_id = gt_df['Prot.ID'].iloc[0] if not gt_df.empty else base_name
        
        results.append({
            'Prot.ID': prot_id,
            'Method': method,
            'TP': tp,
            'FN': fn,
            'TN': tn,
            'FP': fp,
            'Sen': sen,
            'Spe': spe,
            'Acc': acc,
            'Pre': pre,
            'MCC': mcc
        })

# 将所有蛋白的评估结果保存到一个CSV文件中
results_df = pd.DataFrame(results)
output_file = 'evaluation_results.csv'
results_df.to_csv(output_file, index=False)
print(f"所有文件处理完毕，结果已保存到 {output_file}")


所有文件处理完毕，结果已保存到 evaluation_results.csv
